# Autoencoder
This notebook makes use of the **Autoencoder**, which is used to reduce the dimensionality of our dataset in a non-linear way. Furthermore, we then apply **k-means Clustering** as in our last notebook in our new created **Latent Space** in lower dimension. We do so, to get rid of less important variables and achieve a better Clustering.

In [1]:
!pip install -q -r ../../requirements.txt &> /dev/null

In [2]:
from ipywidgets import FloatSlider
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from ipywidgets import interact, IntSlider
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import matplotlib.dates as mdates
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.utils.data import DataLoader, TensorDataset, random_split

import sys
sys.path.append('../..')
import helper_functions     # Own file.
import importlib
importlib.reload(helper_functions)

<module 'helper_functions' from '/home/jovyan/spatiotemporal-mining-medsea/information_filtering/newdata_no/models/../../helper_functions.py'>

In [3]:
SEED = 27

np.random.seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Data Loading & Preprocessing

In [4]:
trend_removal = False

In [5]:
ds_train = xr.open_dataset("../../data/medsea1987to2025_train.nc")

z_temp = helper_functions.preprocessing(ds_train, ["thetao", "so"], [50, 300, 1000], "location", trend_removal, 1)
X_train = z_temp.values.astype(np.float32)
input_dimension = X_train.shape[1]

train_loader = DataLoader(TensorDataset(torch.from_numpy(X_train)), batch_size=32, shuffle=True)

In [6]:
ds_train = xr.open_dataset("../../data/medsea1987to2025_val.nc")

z_temp = helper_functions.preprocessing(ds_train, ["thetao", "so"], [50, 300, 1000], "location", trend_removal, 1)
X_val = z_temp.values.astype(np.float32)
val_dataset = TensorDataset(torch.from_numpy(X_val))

val_size = int(0.4 * len(val_dataset))
_ , val_subset = random_split(
    val_dataset,
    [len(val_dataset) - val_size, val_size],
    generator=torch.Generator().manual_seed(SEED)
)

val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

## The Architecture

In [7]:
class VariationalAutoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim, dropout):
        super().__init__()

        # Encoder for mean and logvar
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.Dropout(dropout),
            nn.LeakyReLU(),

            nn.Linear(512, 32),
            nn.BatchNorm1d(32),
            nn.Dropout(dropout),
            nn.LeakyReLU()
        )
        self.fc_mean = nn.Linear(32, latent_dim)
        self.fc_logvar = nn.Linear(32, latent_dim)

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(),

            nn.Linear(32, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),

            nn.Linear(512, input_dim)
        )

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std

    def forward(self, x):
        h = self.encoder(x)
        z_mean = self.fc_mean(h)
        z_logvar = self.fc_logvar(h)
        z = self.reparameterize(z_mean, z_logvar)
        x_recon = self.decoder(z)
        return x_recon, z_mean, z_logvar

## Training Loop

In [8]:
def train(num_epochs: int, kl_annealing_epochs: int = 50, bint = 100):
    torch.cuda.empty_cache()

    train_losses = []
    val_losses = []
    train_recon_list = []
    train_kl_list = []
    val_recon_list = []
    val_kl_list = []

    for epoch in range(num_epochs):
        beta = min(bint, epoch / kl_annealing_epochs * bint)

        model.train()
        running_train_recon = 0.0
        running_train_kl = 0.0

        for batch in train_loader:
            x = batch[0].to(device).float()

            optimizer.zero_grad()
            x_recon, z_mean, z_logvar = model(x)

            kl_loss = -0.5 * torch.sum(1 + z_logvar - z_mean.pow(2) - z_logvar.exp(), dim=1)
            kl_loss = torch.mean(kl_loss)

            recon_loss = reconstruction_loss_fn(x_recon, x)

            loss = recon_loss + beta * kl_loss
            loss.backward()
            optimizer.step()

            running_train_recon += recon_loss.item() * x.size(0)
            running_train_kl += kl_loss.item() * x.size(0)

        train_recon = running_train_recon / len(train_loader.dataset)
        train_kl = running_train_kl / len(train_loader.dataset)
        train_loss = train_recon + beta * train_kl

        train_losses.append(train_loss)
        train_recon_list.append(train_recon)
        train_kl_list.append(train_kl)

        # Validation
        model.eval()
        running_val_recon = 0.0
        running_val_kl = 0.0

        with torch.no_grad():
            for batch in val_loader:
                x = batch[0].to(device).float()
                x_recon, z_mean, z_logvar = model(x)

                kl_loss = -0.5 * torch.sum(1 + z_logvar - z_mean.pow(2) - z_logvar.exp(), dim=1)
                kl_loss = torch.mean(kl_loss)

                recon_loss = reconstruction_loss_fn(x_recon, x)

                running_val_recon += recon_loss.item() * x.size(0)
                running_val_kl += kl_loss.item() * x.size(0)

        val_recon = running_val_recon / len(val_loader.dataset)
        val_kl = running_val_kl / len(val_loader.dataset)
        val_loss = val_recon + beta * val_kl

        val_losses.append(val_loss)
        val_recon_list.append(val_recon)
        val_kl_list.append(val_kl)

        if (epoch+1) % 10 == 0:
            print(
                f"Epoch {epoch+1}/{num_epochs} | "
                f"β: {beta:.3f} | "
                f"Train Loss: {train_loss:.4f} (Recon: {train_recon:.4f}, KL: {train_kl:.4f}) | "
                f"Val Loss: {val_loss:.4f} (Recon: {val_recon:.4f}, KL: {val_kl:.4f})"
            )

    return train_losses, val_losses, train_recon_list, val_recon_list, train_kl_list, val_kl_list

## Setup

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Training using device: {device}')

model = VariationalAutoencoder(input_dim=input_dimension, latent_dim=3, dropout=0.2).to(device)
model = model.float()

summary(model, input_size=(1, X_train.shape[1]))

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
reconstruction_loss_fn = nn.MSELoss(reduction='mean')

Training using device: cuda


In [10]:
train_losses, val_losses, train_recons, val_recons, train_kls, val_kls = train(800,200, 0.05)

Epoch 10/800 | β: 0.002 | Train Loss: 0.6868 (Recon: 0.6815, KL: 2.3280) | Val Loss: 0.6344 (Recon: 0.6289, KL: 2.4220)
Epoch 20/800 | β: 0.005 | Train Loss: 0.6477 (Recon: 0.6343, KL: 2.8201) | Val Loss: 0.6063 (Recon: 0.5937, KL: 2.6680)
Epoch 30/800 | β: 0.007 | Train Loss: 0.6287 (Recon: 0.6045, KL: 3.3330) | Val Loss: 0.5887 (Recon: 0.5672, KL: 2.9623)
Epoch 40/800 | β: 0.010 | Train Loss: 0.6344 (Recon: 0.6007, KL: 3.4629) | Val Loss: 0.5665 (Recon: 0.5370, KL: 3.0213)
Epoch 50/800 | β: 0.012 | Train Loss: 0.6150 (Recon: 0.5703, KL: 3.6495) | Val Loss: 0.5683 (Recon: 0.5307, KL: 3.0684)
Epoch 60/800 | β: 0.015 | Train Loss: 0.6152 (Recon: 0.5622, KL: 3.5950) | Val Loss: 0.5506 (Recon: 0.5037, KL: 3.1810)
Epoch 70/800 | β: 0.017 | Train Loss: 0.6148 (Recon: 0.5548, KL: 3.4785) | Val Loss: 0.5495 (Recon: 0.4941, KL: 3.2151)
Epoch 80/800 | β: 0.020 | Train Loss: 0.6009 (Recon: 0.5338, KL: 3.3983) | Val Loss: 0.5457 (Recon: 0.4848, KL: 3.0843)
Epoch 90/800 | β: 0.022 | Train Loss: 0.

KeyboardInterrupt: 

## Evaluation

In [11]:
helper_functions.plot_metrics([(train_losses, "Train Loss"), (val_losses, "Validation Loss")], "Loss")
helper_functions.plot_metrics([(train_recons, "Train MSE-Scores"), (val_recons, "Validation MSE-Scores")], "MSE Score")
helper_functions.plot_metrics([(train_kls, "Train KL-Scores"), (val_kls, "Validation KL-Scores")], "KL-Score")

NameError: name 'train_losses' is not defined

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

latents_mu = []
latents_logvar = []

with torch.no_grad():
    for batch in train_loader:
        x = batch[0].to(device).float()
        _, mu, logvar = model(x)
        latents_mu.append(mu.cpu())
        latents_logvar.append(logvar.cpu())

mu_all = torch.cat(latents_mu, dim=0)           # shape: (n_samples, latent_dim)
logvar_all = torch.cat(latents_logvar, dim=0)   # shape: (n_samples, latent_dim)

# Statistics
mu_std = mu_all.std(dim=0)
logvar_mean = logvar_all.mean(dim=0)
logvar_std = logvar_all.std(dim=0)

print("Std of mu per latent dim:")
print(mu_std)

print("\nMean of logvar per latent dim:")
print(logvar_mean)

print("\nStd of logvar per latent dim:")
print(logvar_std)

Std of mu per latent dim:
tensor([0.9254, 0.9531, 1.2836])

Mean of logvar per latent dim:
tensor([-1.3198, -1.4223, -1.9194])

Std of logvar per latent dim:
tensor([0.3368, 0.1840, 0.3913])


In [13]:
torch.save(model.state_dict(), "VAE.pth")